## Welcom to Griffin

### Introduction

Griffin MMM is a Media Mix Modeling solution designed to empower marketers with advanced analytics and intelligent insights. As part of an evolving suite of tools, Griffin MMM stands at the forefront of marketing technology, enabling users to optimize their strategies across various channels effectively.

At its core, Griffin MMM is a powerful analytical tool that helps navigate the complex marketing landscape. It provides a robust framework for analyzing the effectiveness of different marketing channels, allowing marketers to make data-driven decisions and maximize their return on investment (ROI).


### What is MMM?

Marketing mix modeling (MMM) is a privacy-friendly, highly resilient, data-driven statistical analysis that quantifies the incremental sales impact and ROI of marketing and non-marketing activities.

MMM is an econometric model that aims to quantify the incremental impact of marketing and non-marketing activities on a pre-defined KPI (like sales or website visits). This is a holistic model used to understand how to allocate a marketing budget across marketing channels, products and regions and can help forecast the impact of future events or campaigns:

<img src="https://facebookexperimental.github.io/Robyn/img/howMMMisBuilt.png" width="600"/>

## Setup

In [ ]:
# downloads docs/ folder from demo repo
# downloads demo dataset/config/holidays
# installs griffin-mmm-demo package.
!pip install -q -U git+https://@github.com/griffin-analytics/griffin-mmm-demo.git
!pip install ipywidgets


Short description of files in docs/ folder (configuration guide, documentation pdf, quickstart, output guide, etc.)

## Demo

In [ ]:
from base_driver import MMMBaseDriver
from base_driver import utils as ut
from mmm import describe as dsc
from mmm import plot as mplt

### 1. Inputs

#### Short description of input files

config.yml<br>
input_data.csv/xsls<br>
(optional) holidays.xsls<br>

Dataset Overview

- Interactive data preview (NEW)
- Sample dataset exploration
- Data quality checks

<hr>
Configuration

- Interactive config builder (Config file will be edited using GUI widgets (based on IPyWidgets)
- Parameter explanation
    - media variables
    - extra variables
    - number of iters/rounds/etc.
- Best practices guide

In [ ]:
import ipywidgets as widgets

In [1]:
print('test')

test


In [3]:
import ipywidgets as widgets
from IPython.display import display, HTML
import yaml

class GridConfigWidget:
    def __init__(self):
        # Style
        self.style = HTML("""
        <style>
        .widget-label { font-size: 0.9em; }
        .header { 
            font-weight: bold;
            color: #2c3e50;
            margin: 5px 0;
        }
        </style>
        """)
        
        # Data Handling & Basic Settings
        self.raw_data_granularity = widgets.Dropdown(
            options=['daily', 'weekly'],
            value='weekly',
            description='Frequency:',
            layout=widgets.Layout(width='200px')
        )
        
        self.train_test_ratio = widgets.BoundedFloatText(
            value=1.0, min=0.5, max=1.0,
            description='Train Ratio:',
            layout=widgets.Layout(width='200px')
        )
        
        self.date_col = widgets.Text(
            value='date',
            description='Date Col:',
            layout=widgets.Layout(width='200px')
        )
        
        self.target_col = widgets.Text(
            value='subscribers',
            description='Target Col:',
            layout=widgets.Layout(width='200px')
        )
        
        # Model Parameters
        self.tune = widgets.IntText(
            value=1000,
            description='Tune:',
            layout=widgets.Layout(width='200px')
        )
        
        self.draws = widgets.IntText(
            value=2000,
            description='Draws:',
            layout=widgets.Layout(width='200px')
        )
        
        self.chains = widgets.IntText(
            value=4,
            description='Chains:',
            layout=widgets.Layout(width='200px')
        )
        
        self.ad_stock_max_lag = widgets.IntText(
            value=8,
            description='Max Lag:',
            layout=widgets.Layout(width='200px')
        )

        # Media Channels (in grid)
        self.media_channels = widgets.GridspecLayout(4, 3, width='650px')
        for i in range(4):
            self.media_channels[i, 0] = widgets.Text(
                value=f'Media Channel {i+1}',
                description=f'Ch {i+1}:',
                layout=widgets.Layout(width='200px')
            )
            self.media_channels[i, 1] = widgets.Text(
                value=f'media_imp_{i+1}',
                description='Imp:',
                layout=widgets.Layout(width='200px')
            )
            self.media_channels[i, 2] = widgets.Text(
                value=f'media_cost_{i+1}',
                description='Cost:',
                layout=widgets.Layout(width='200px')
            )

        # Prophet Settings (as checkboxes in a row)
        self.prophet_settings = widgets.GridspecLayout(1, 5, width='650px')
        self.prophet_settings[0, 0] = widgets.Checkbox(value=True, description='Holidays')
        self.prophet_settings[0, 1] = widgets.Checkbox(value=True, description='Yearly')
        self.prophet_settings[0, 2] = widgets.Checkbox(value=True, description='Weekly')
        self.prophet_settings[0, 3] = widgets.Checkbox(value=True, description='Trend')
        self.prophet_settings[0, 4] = widgets.Text(
            value='US',
            description='Country:',
            layout=widgets.Layout(width='120px')
        )

        # Features in a single line
        self.features = widgets.Text(
            value='covid_index,competitor_spend,promo_events',
            description='Features:',
            layout=widgets.Layout(width='650px')
        )
        
        self.ignore_cols = widgets.Text(
            value='price,other_events',
            description='Ignore:',
            layout=widgets.Layout(width='650px')
        )

        # Save Button and Output
        self.save_button = widgets.Button(
            description='Save Config',
            button_style='success',
            layout=widgets.Layout(width='100px')
        )
        self.save_button.on_click(self.save_config)
        self.output = widgets.Output()

    def save_config(self, b):
        with self.output:
            self.output.clear_output()
            config = {
                'raw_data_granularity': self.raw_data_granularity.value,
                'train_test_ratio': self.train_test_ratio.value,
                'ignore_cols': [x.strip() for x in self.ignore_cols.value.split(',')],
                'date_col': self.date_col.value,
                'target_col': self.target_col.value,
                'extra_features_cols': [x.strip() for x in self.features.value.split(',')],
                'extra_features_impact': {'competitor_spend': 'negative'},
                'media': [
                    {
                        'display_name': self.media_channels[i, 0].value,
                        'impressions_col': self.media_channels[i, 1].value,
                        'spend_col': self.media_channels[i, 2].value
                    }
                    for i in range(4)
                ],
                'tune': self.tune.value,
                'draws': self.draws.value,
                'chains': self.chains.value,
                'ad_stock_max_lag': self.ad_stock_max_lag.value,
                'prophet': {
                    'include_holidays': self.prophet_settings[0, 0].value,
                    'yearly_seasonality': self.prophet_settings[0, 1].value,
                    'weekly_seasonality': self.prophet_settings[0, 2].value,
                    'trend': self.prophet_settings[0, 3].value,
                    'holiday_country': self.prophet_settings[0, 4].value
                }
            }
            
            with open('config.yaml', 'w') as f:
                yaml.dump(config, f, default_flow_style=False)
            print("Saved to config.yaml")

    def display(self):
        basic_settings = widgets.GridspecLayout(2, 4, width='850px')
        basic_settings[0, 0] = self.raw_data_granularity
        basic_settings[0, 1] = self.train_test_ratio
        basic_settings[0, 2] = self.date_col
        basic_settings[0, 3] = self.target_col
        basic_settings[1, 0] = self.tune
        basic_settings[1, 1] = self.draws
        basic_settings[1, 2] = self.chains
        basic_settings[1, 3] = self.ad_stock_max_lag

        display(widgets.VBox([
            widgets.HTML('<h3>Basic Settings</h3>'),
            basic_settings,
            widgets.HTML('<h3>Features</h3>'),
            self.features,
            self.ignore_cols,
            widgets.HTML('<h3>Media Channels</h3>'),
            self.media_channels,
            widgets.HTML('<h3>Prophet Settings</h3>'),
            self.prophet_settings,
            self.save_button,
            self.output
        ]))

# Create and display widget
config_widget = GridConfigWidget()
config_widget.display()

Pro vs Demo Comparison

- Feature comparison matrix
- Upgrade benefits

In [ ]:
from base_driver import MMMBaseDriver
from base_driver import utils as ut
from mmm import describe as dsc
from mmm import plot as mplt

driver = MMMBaseDriver(
    "demo_config.yml",
    "demo_data.xlsx",
    "holidays.xlsx")

### 2. Preprocessing

Interactive Data Validation (NEW)

- Automated data quality checks
- Missing value analysis
- Outlier detection

In [ ]:
ut.setup_logger()
ut.set_style()

In [ ]:
driver.check_quality()

Output Interpretation

- Interactive visualization dashboard (NEW)
- Key metrics explanation
- Benchmark comparisons

In [ ]:
high_var = driver.highlight_variances()
high_inf = driver.highlight_high_vif_values()
high_spf = driver.highlight_low_spend_fractions()
fig, corr_df = driver.plot_correlation()
all_mds_fig = driver.plot_all_media_spend()

### 3. Run model

Execution Monitor

- Progress tracking
- Resource usage monitoring
- Error handling and recovery

In [ ]:
driver.main()

Results Dashboard (NEW)

- Real-time performance metrics
- Comparative analysis
- Export capabilities

In [ ]:
driver.compute_elpd(model_name=driver.run_id, results_dir="/content/results")

Let's save posterior samples into the results folder

In [ ]:
driver.save_posterior_samples(results_dir='/content/results')

Plot the posterior distributions. How to interpret the plot? What conclusions you can make? Based on what?

In [ ]:
driver.plot_posterior_distributions(results_dir='/content/results')

Maybe make each each plot on a separate cell?<br>
Add plot descriptions and give some best practices:
- Model structure plot
- Model trace plot
- R^2 score
- Posterior predictive plot
- Components contributions plot
- Waterfall plot

In [ ]:
mdl_st = driver.plot_model_structure()
model_trace = driver.plot_model_trace()
r2_score = driver.calculate_train_r_squared()
posterior_predictive = driver.plot_posterior_predictive()
components_contribution = driver.plot_components_contributions()
waterfall_plot = driver.plot_waterfall_components_decomposition()

### 4. Check results. Diagnostics.

Interactive Diagnostic Dashboard (NEW)

- Key performance indicators
- Model health metrics
- Warning indicators


Visualization Suite

- Interactive plots with tooltips
- Customizable views
- Export options


Interpretation Guide

- Common patterns explanation
- Troubleshooting suggestions
- Best practices

In [ ]:
quick_stats = dsc.quick_stats(driver.model)

In [ ]:
waterfall_plot

In [ ]:
weekly_spend_curve = mplt.weekly_spend_by_channel(driver.model)

In [ ]:
high_var

In [ ]:
high_inf

In [ ]:
high_spf

In [ ]:
fig

In [ ]:
corr_df

In [ ]:
all_mds_fig

In [ ]:
mdl_st

In [ ]:
model_trace

In [ ]:
driver.display_image("weekly_media_and_baseline_contribution.png")

In [ ]:
driver.display_image("weekly_media_contribution.png")

In [ ]:
r2_score

In [ ]:
driver.per_observation_df

In [ ]:
posterior_predictive

In [ ]:
components_contribution

In [ ]:
driver.data_to_fit.to_data_frame()

In [ ]:
quick_stats

In [ ]:
weekly_spend_curve

In [ ]:
model = driver.model
model.plot_direct_contribution_curves(
    show_fit=True,
    method="sigmoid",
    export_curves=True,  # save curves as numerical outputs
    results_dir='./results'
)

In [ ]:
driver.display_image("weekly_media_and_baseline_contribution.png")

## Budget Optimization.

What is Budget Optimization? 
<br>
<br>
How we do it?

Optimization Setup

- Parameter selection interface
- Constraint definition
- Goal setting

In [ ]:
from optimization_file_py import optimize_marketing_budget

Results Analysis

- Performance comparison
- ROI calculation (NEW)
- Recommendation engine

In [ ]:
optimize_marketing_budget(driver.model, driver.processed_data, driver.config, driver.results_dir)

## Export resutls

One-Click Export Options (NEW)

- Results download
- Configuration export
- Report generation

In [ ]:
# Define the list of files to exclude from the zip
excluded_files = [
    '.config',      # Exclude .config folder
    '.ipynb_checkpoints',  # Exclude .ipynb_checkpoints folder
    'data',         # Exclude data folder
    'drive',        # Exclude Google Drive folder
    'sample_data',  # Exclude sample data folder
    #'results/model.nc',  # Exclude model file
    #'holidays.xlsx',  # Exclude specific file
    'results/trace.nc'  # Exclude trace file
]

from base_driver import utils as ut # This ut has create_downloadable_zip
ut.create_downloadable_zip(excluded_files) # Now call the function to create and download the zip

## Next steps

Pro Version Benefits

- Feature comparison
- ROI calculator
- Upgrade path

## Support & Resources

Help Center

- FAQ
- Troubleshooting guide
- Video tutorials
<hr>

Contact Information

- Support channels
- Sales inquiry
- Documentation resources
<hr>

Legal Information

- Copyright notice
- Terms of use
- Privacy policy